In [1]:
"""IMPORTANT CELLS MUST BE RUN IN ORDER
    2018 Nationals
"""
import os

from openpyxl import load_workbook
import pandas as pd

from lifter_api import LifterAPI

In [2]:
# authenticate

URL = None
# URL = "http://localhost:8000"

auth_token = os.getenv("LOCAL_API_TOKEN")
api = LifterAPI(url=URL, auth_token=auth_token)
if URL != "http://localhost:8000":
    auth_token = os.getenv("API_TOKEN")
    api = LifterAPI(url="https://api.lifter.shivan.xyz", auth_token=auth_token)
    print("connected LIVE")
else:
    print("connected locally")

connected LIVE


In [3]:
# Create competition and the sessions
# This will also check if the competition already exists by name, and also sessions by name as well.
def create_competition(date_start, date_end, competition_name, location):
    # search for a competition
    current_competitions = {}
    next_page = "page=1"
    while next_page:
        competitions = api.competitions(page=int(next_page.split("page")[-1].replace("=", "")))
        for competition in competitions['results']:
            current_competitions[competition['competition_name']] = competition['reference_id']
        next_page = competitions['next']

    current_competition_names = list(current_competitions.keys())
    
    if competition_name not in current_competition_names:
        print(f"{competition_name} created.")
        return api.create_competition(date_start=date_start,
                            date_end=date_end,
                            location=location,
                            competition_name=competition_name)
    print(f"ERROR: {competition_name} already exists!")
    return api.get_competition(competition_id=current_competitions[competition_name])

    

def create_session(competition_id, session_datetime, referee_first, referee_second, referee_third, technical_controller, jury, marshall, announcer, timekeeper):
    # search for sessions
    sessions = api.sessions(competition_id=competition_id)
    current_sessions = {}
    for session in sessions['results']:
        current_sessions[session['session_datetime']] = session['reference_id']
    
    current_session_datetimes = list(current_sessions.keys())
    
    if session_datetime not in current_session_datetimes:
        print(f"Session at {session_datetime} created.")
        return api.create_session(
            competition_id=competition_id,
            session_datetime=session_datetime,
            referee_first=referee_first,
            referee_second=referee_second,
            referee_third=referee_third,
            technical_controller=technical_controller,
            jury=jury,
            marshall=marshall,
            announcer=announcer,
            timekeeper=timekeeper,
        )
    print(f"ERROR: Session at {session_datetime} already exists.")
    return api.get_session(competition_id=competition_id, session_id=current_sessions[session_datetime])


In [4]:

# create athletes

def create_athlete(first_name: str, last_name: str, yearborn: int):
    first_name = first_name
    last_name = last_name
    if api.find_athlete(search=(first_name + " " + last_name))["count"] == 0:
        create = api.create_athlete(
            first_name=first_name, last_name=last_name, yearborn=yearborn
        )
        print(f"athlete created {create['first_name']} {create['last_name']}")
    else:
        print("athlete already exists")

In [5]:

def give_lift_status(weight: str):
    if weight == "---":
        return ("DNA", 0)
    if weight >  0:
        return ("LIFT", weight)
    return ("NOLIFT", abs(weight))

def parse_lift(lift: list[list[str]], weight_category: str):
    lift_data = { 
    "athlete_name": lift[1],
    "snatch_first": give_lift_status(lift[5])[0],
    "snatch_first_weight": int(give_lift_status(lift[5])[1]),
    "snatch_second": give_lift_status(lift[6])[0],
    "snatch_second_weight": int(give_lift_status(lift[6])[1]),
    "snatch_third": give_lift_status(lift[7])[0],
    "snatch_third_weight": int(give_lift_status(lift[7])[1]),
    "cnj_first": give_lift_status(lift[8])[0],
    "cnj_first_weight": int(give_lift_status(lift[8])[1]),
    "cnj_second": give_lift_status(lift[9])[0],
    "cnj_second_weight": int(give_lift_status(lift[9])[1]),
    "cnj_third": give_lift_status(lift[10])[0],
    "cnj_third_weight": int(give_lift_status(lift[10])[1]),
    "bodyweight": float(lift[4]),
    "weight_category": weight_category,
    "team": str(lift[3]),
    "lottery_number": int(lift[0]),
    }
    return lift_data 

def create_lift(competition_id: str, session_id: str, lift: list[dict[str: str|int]]):
    print(lift)
    first_name = lift["athlete_name"].split(" ")[0]
    last_name = lift["athlete_name"].split(" ")[1]
    athlete = api.find_athlete(search=(first_name + " " + last_name))
    print(athlete['results'][0]['reference_id'])
    if athlete['count'] == 1:
        api.create_lift(
            competition_id=competition_id,
            session_id=session_id,
            athlete_id=athlete['results'][0]['reference_id'],
            snatch_first=lift['snatch_first'],
            snatch_first_weight=lift['snatch_first_weight'],
            snatch_second=lift['snatch_second'],
            snatch_second_weight=lift['snatch_second_weight'],
            snatch_third=lift['snatch_third'],
            snatch_third_weight=lift['snatch_third_weight'],
            cnj_first=lift['cnj_first'],
            cnj_first_weight=lift['cnj_first_weight'],
            cnj_second=lift['cnj_second'],
            cnj_second_weight=lift['cnj_second_weight'],
            cnj_third=lift['cnj_third'],
            cnj_third_weight=lift['cnj_third_weight'],
            bodyweight=lift['bodyweight'],
            weight_category=lift['weight_category'],
            team=lift['team'],
            lottery_number=lift['lottery_number'],
            )
        print("Lift created successfully")
    elif athlete['count'] > 1:
        print("Multiple athlete with the name query exist")
    else:
        print(f"No athlete of the name, {lift['athlete_name']} this name exist")


def create_athlete_lift(competition_id: str, session_id: str, lift: dict, weight_category: str):
        
    create_athlete(
        first_name=lift[1].split(" ")[0],
        last_name=lift[1].split(" ")[1],
        yearborn=lift[2],
    )

    create_lift(competition_id=competition_id, session_id=session_id, lift=parse_lift(lift=lift, weight_category=weight_category))

In [7]:

competition_data = {
    "date_start": "2018-09-21",
    "date_end": "2018-09-23",
    "location": "Kolmar, Papatoetoe, Auckland",
    "competition_name": "2018 Junior Senior National Championships",
}

# create competition
new_competition = create_competition(
    date_start=competition_data['date_start'],
    date_end=competition_data['date_end'],
    location=competition_data['location'],
    competition_name=competition_data['competition_name']
)

competition_id = new_competition['reference_id']

ERROR: 2018 Junior Senior National Championships already exists!


In [11]:
# get file data into 

file = "2018.national_championships.xlsx"

wb = load_workbook(filename=file)
sheet = wb.sheetnames

sessions = {}

for session in wb.sheetnames:
    sessions[session] = pd.read_excel(file, sheet_name=session)

list(wb.sheetnames)

['Session 1 ',
 'Session 2 ',
 'Session 3',
 'Session 4',
 'Session 5',
 'Session 6',
 'Session 7',
 'Session 8',
 'Session 9',
 'Session 10']

In [42]:
df = sessions["Session 1 "]
df

,2018 Junior Senior National Championships,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Held:,2018-09-21 00:00:00,NaN,Session #,1,NaN,NaN,at:,"Kolmar, Papatoetoe, Auckland",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Start,NaN,NaN,NaN,Body,SNATCH,NaN,NaN,CLEAN & JERK,NaN,NaN,Best,NaN,NaN,Sinclair,NaN
2,#,Name,Born,Club,weight,1,2,3,1,2.0,3.0,Snatch,C & J,TOTAL,Score,Place
3,1,Sachiko Kawakami,1969,GUEST,51.7,47,49,-51,58,-60.0,60.0,49,60,109,163.214684,GUEST
4,2,Anja Jennings,2003,KKL,50.3,49,-52,53,58,63.0,64.0,53,64,117,178.841324,4
5,3,Ruby Garbutt,1992,CCW,52,50,-52,-53,62,65.0,-69.0,50,65,115,171.463955,5
6,4,Charlotte Moss,1995,NSOW,52.3,62,66,-69,80,-85.0,-85.0,66,80,146,216.765726,1
7,5,Kiri Sharp,1987,OWL,52.2,48,50,-52,62,-64.0,-64.0,50,62,112,166.519821,6
8,6,Rebecca Jans,1988,INDEP,52.5,54,56,58,70,72.0,74.0,58,74,132,195.433095,2
9,7,Sheryl Tan,1988,SHQ,51.8,-49,49,51,60,-64.0,67.0,51,67,118,176.438435,3


In [43]:
session_data = {
    'session_datetime': "2018-09-21T07:00:00+13:00",
    'referee_first': "Paule Poulin",
    'referee_second': "Barbara Greive",
    'referee_third': "Jenni Ebert",
    'technical_controller': "David Timajo",
    'jury': "None",
    'marshall': "Jason Fanning",
    'announcer': "Keisha-Dean Soffee",
    'timekeeper': "Keisha-Dean Soffee",
}

new_session = create_session(
    competition_id=new_competition['reference_id'],
    session_datetime=session_data['session_datetime'],
    referee_first=session_data['referee_first'],
    referee_second=session_data['referee_second'],
    referee_third=session_data['referee_third'],
    technical_controller=session_data['technical_controller'],
    jury=session_data['jury'],
    marshall=session_data['marshall'],
    announcer=session_data['announcer'],
    timekeeper=session_data['timekeeper'],
)

session_id = new_session["reference_id"]

Session at 2018-09-21T07:00:00+13:00 created.


In [47]:
# Under 53

weight_category = "W53"
session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[3:10]:
    print(lift[1])
if input("Test? [Y]") == "N":
    for lift in lifts[3:10]:
        create_athlete_lift(
            competition_id=competition_id,
            session_id=session_id,
            lift=lift,
            weight_category=weight_category,
            )

Sachiko Kawakami
Anja Jennings
Ruby Garbutt
Charlotte Moss
Kiri Sharp
Rebecca Jans
Sheryl Tan
athlete already exists
{'athlete_name': 'Sachiko Kawakami', 'snatch_first': 'LIFT', 'snatch_first_weight': 47, 'snatch_second': 'LIFT', 'snatch_second_weight': 49, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 51, 'cnj_first': 'LIFT', 'cnj_first_weight': 58, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 60, 'cnj_third': 'LIFT', 'cnj_third_weight': 60, 'bodyweight': 51.7, 'weight_category': 'W53', 'team': 'GUEST', 'lottery_number': 1}
DAwMBYy
Lift created successfully
athlete already exists
{'athlete_name': 'Anja Jennings', 'snatch_first': 'LIFT', 'snatch_first_weight': 49, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 52, 'snatch_third': 'LIFT', 'snatch_third_weight': 53, 'cnj_first': 'LIFT', 'cnj_first_weight': 58, 'cnj_second': 'LIFT', 'cnj_second_weight': 63, 'cnj_third': 'LIFT', 'cnj_third_weight': 64, 'bodyweight': 50.3, 'weight_category': 'W53', 'team': 'KKL', 'lottery_number': 2}

In [48]:
# Under 58

weight_category = "W58"
session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[11:22]:
    create_athlete_lift(
        competition_id=competition_id,
        session_id=session_id,
        lift=lift,
        weight_category=weight_category,
        )
# for lift in lifts[11:22]:
#     print(lift[1])

athlete created Lucy Anne
{'athlete_name': 'Lucy Anne Thompson', 'snatch_first': 'NOLIFT', 'snatch_first_weight': 45, 'snatch_second': 'LIFT', 'snatch_second_weight': 45, 'snatch_third': 'LIFT', 'snatch_third_weight': 49, 'cnj_first': 'LIFT', 'cnj_first_weight': 55, 'cnj_second': 'LIFT', 'cnj_second_weight': 60, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 64, 'bodyweight': 56.7, 'weight_category': 'W58', 'team': '41DW', 'lottery_number': 8}
nr3ykNM
Lift created successfully
athlete created Leigh Martel
{'athlete_name': 'Leigh Martel', 'snatch_first': 'LIFT', 'snatch_first_weight': 55, 'snatch_second': 'LIFT', 'snatch_second_weight': 58, 'snatch_third': 'LIFT', 'snatch_third_weight': 60, 'cnj_first': 'LIFT', 'cnj_first_weight': 66, 'cnj_second': 'LIFT', 'cnj_second_weight': 69, 'cnj_third': 'LIFT', 'cnj_third_weight': 72, 'bodyweight': 57.9, 'weight_category': 'W58', 'team': 'OWL', 'lottery_number': 9}
brJlDNG
Lift created successfully
athlete created Natasha Cranmer
{'athlete_name': 'Na

In [12]:
df = sessions['Session 2 ']
df

,2018 Junior Senior National Championships,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Held:,2018-09-21 00:00:00,NaN,Session #,2,NaN,NaN,at:,"Kolmar, Papatoetoe, Auckland",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Start,NaN,NaN,NaN,Body,SNATCH,NaN,NaN,CLEAN & JERK,NaN,NaN,Best,NaN,NaN,Sinclair,NaN
2,#,Name,Born,Club,weight,1,2,3,1,2.0,3.0,Snatch,C & J,TOTAL,Score,Place
3,NaN,Men 56kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Avery Fesolai,2002,SHQ,54.7,62,65,68,82,86.0,89.0,68,89,157,244.714842,1
5,2,Elian Hoffman,2002,CSI,54.6,-63,63,67,80,85.0,-89.0,67,85,152,237.251966,2
6,NaN,Men 62kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3,Jiro Ishida,1990,OWL,61.6,85,90,-94,100,-105.0,107.0,90,107,197,281.801639,NaN
8,4,Kieren Hempsey,1999,NSOW,61.7,79,83,86,102,108.0,112.0,86,112,198,282.91835,NaN
9,NaN,Men 69kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
session_data = {
    'session_datetime': "2018-09-21T09:00:00+12:00",
    'referee_first': "Tony Ebert",
    'referee_second': "Garry Marshall",
    'referee_third': "Barbara Grieve",
    'technical_controller': "Odette Smith",
    'jury': "None",
    'marshall': "Mayy Dyson",
    'announcer': "Ray Everest",
    'timekeeper': "Ray Everest",
}

new_session = create_session(
    competition_id=new_competition['reference_id'],
    session_datetime=session_data['session_datetime'],
    referee_first=session_data['referee_first'],
    referee_second=session_data['referee_second'],
    referee_third=session_data['referee_third'],
    technical_controller=session_data['technical_controller'],
    jury=session_data['jury'],
    marshall=session_data['marshall'],
    announcer=session_data['announcer'],
    timekeeper=session_data['timekeeper'],
)

session_id = new_session["reference_id"]

Session at 2018-09-21T09:00:00+12:00 created.


In [16]:

# Under M56

weight_category = "M56"
start = 4
end = 6
# session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[start:end]:
    print(lift[1])
if input("Test? [Y]") == "N":
    for lift in lifts[start:end]:
        create_athlete_lift(
            competition_id=competition_id,
            session_id=session_id,
            lift=lift,
            weight_category=weight_category,
            )

Avery Fesolai
Elian Hoffman
athlete created Avery Fesolai
{'athlete_name': 'Avery Fesolai', 'snatch_first': 'LIFT', 'snatch_first_weight': 62, 'snatch_second': 'LIFT', 'snatch_second_weight': 65, 'snatch_third': 'LIFT', 'snatch_third_weight': 68, 'cnj_first': 'LIFT', 'cnj_first_weight': 82, 'cnj_second': 'LIFT', 'cnj_second_weight': 86, 'cnj_third': 'LIFT', 'cnj_third_weight': 89, 'bodyweight': 54.7, 'weight_category': 'M56', 'team': 'SHQ', 'lottery_number': 1}
vAEynAE
Lift created successfully
athlete created Elian Hoffman
{'athlete_name': 'Elian Hoffman', 'snatch_first': 'NOLIFT', 'snatch_first_weight': 63, 'snatch_second': 'LIFT', 'snatch_second_weight': 63, 'snatch_third': 'LIFT', 'snatch_third_weight': 67, 'cnj_first': 'LIFT', 'cnj_first_weight': 80, 'cnj_second': 'LIFT', 'cnj_second_weight': 85, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 89, 'bodyweight': 54.6, 'weight_category': 'M56', 'team': 'CSI', 'lottery_number': 2}
8YV2pYG
Lift created successfully


In [19]:

# Under M56

weight_category = "M62"
start = 7
end = 9
# session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[start:end]:
    print(lift[1])
if input("Test? [Y]") == "N":
    for lift in lifts[start:end]:
        create_athlete_lift(
            competition_id=competition_id,
            session_id=session_id,
            lift=lift,
            weight_category=weight_category,
            )

Jiro Ishida
Kieren Hempsey
athlete created Jiro Ishida
{'athlete_name': 'Jiro Ishida', 'snatch_first': 'LIFT', 'snatch_first_weight': 85, 'snatch_second': 'LIFT', 'snatch_second_weight': 90, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 94, 'cnj_first': 'LIFT', 'cnj_first_weight': 100, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 105, 'cnj_third': 'LIFT', 'cnj_third_weight': 107, 'bodyweight': 61.6, 'weight_category': 'M62', 'team': 'OWL', 'lottery_number': 3}
LN0QJY0
Lift created successfully
athlete created Kieren Hempsey
{'athlete_name': 'Kieren Hempsey', 'snatch_first': 'LIFT', 'snatch_first_weight': 79, 'snatch_second': 'LIFT', 'snatch_second_weight': 83, 'snatch_third': 'LIFT', 'snatch_third_weight': 86, 'cnj_first': 'LIFT', 'cnj_first_weight': 102, 'cnj_second': 'LIFT', 'cnj_second_weight': 108, 'cnj_third': 'LIFT', 'cnj_third_weight': 112, 'bodyweight': 61.7, 'weight_category': 'M62', 'team': 'NSOW', 'lottery_number': 4}
lrMZXAy
Lift created successfully


In [21]:

weight_category = "M69"
start = 10
end = 15
# session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[start:end]:
    print(lift[1])
if input("Test? [Y]") == "N":
    for lift in lifts[start:end]:
        create_athlete_lift(
            competition_id=competition_id,
            session_id=session_id,
            lift=lift,
            weight_category=weight_category,
            )

Jayden Double
Ehsan Bayati
Robert De Mayo
Owen Webb
Danyon Sievers
athlete created Jayden Double
{'athlete_name': 'Jayden Double', 'snatch_first': 'LIFT', 'snatch_first_weight': 79, 'snatch_second': 'LIFT', 'snatch_second_weight': 82, 'snatch_third': 'LIFT', 'snatch_third_weight': 84, 'cnj_first': 'NOLIFT', 'cnj_first_weight': 92, 'cnj_second': 'LIFT', 'cnj_second_weight': 93, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 101, 'bodyweight': 68.5, 'weight_category': 'M69', 'team': 'NSOW', 'lottery_number': 5}
2rd0krw
Lift created successfully
athlete created Ehsan Bayati
{'athlete_name': 'Ehsan Bayati', 'snatch_first': 'LIFT', 'snatch_first_weight': 83, 'snatch_second': 'LIFT', 'snatch_second_weight': 87, 'snatch_third': 'LIFT', 'snatch_third_weight': 90, 'cnj_first': 'LIFT', 'cnj_first_weight': 105, 'cnj_second': 'LIFT', 'cnj_second_weight': 110, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 115, 'bodyweight': 67.1, 'weight_category': 'M69', 'team': 'GUEST', 'lottery_number': 6}
KYB04Am
Lift

In [23]:
df = sessions["Session 3"]
df

,2018 National Championships,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Held:,2018-09-21 00:00:00,NaN,Session #,3,NaN,NaN,at:,"Kolmar, Papatoetoe, Auckland",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Start,NaN,NaN,NaN,Body,SNATCH,NaN,NaN,CLEAN & JERK,NaN,NaN,Best,NaN,NaN,Sinclair,NaN
2,#,Name,Born,Club,weight,1,2,3,1,2,3,Snatch,C & J,TOTAL,Score,Place
3,NaN,Women 63kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Paule Poulin,1970,GUEST,61.2,55,-57,-58,-75,-75,-75,55,---,NaN,NaN,NaN
5,2,Sienna Fesolai,2005,SHQ,62.3,60,-63,-65,76,79,82,60,82,142,187.378488,7
6,3,Emma McIntyre,1991,INDEP,62.6,68,-73,73,83,-86,86,73,86,159,209.194471,2
7,4,Isobel Ferguson,1995,BOP,62.2,-53,53,55,75,-78,-78,55,75,130,171.713219,9
8,5,Elizabeth Bateman,1991,OBC,62.8,68,-72,-72,86,-90,-90,68,86,154,202.222167,4
9,6,Samara Wright,2001,41DW,62.7,65,-68,-70,85,-88,88,65,88,153,201.104263,5


In [24]:
session_data = {
    'session_datetime': "2018-09-21T15:00:00+12:00",
    'referee_first': "Jenni Ebert",
    'referee_second': "Saxon Gregory-Hunt",
    'referee_third': "Keisha Dean Soffee",
    'technical_controller': "Barbara Greive",
    'jury': "None",
    'marshall': "Kelsie Chapman",
    'announcer': "Gabrielle Peach",
    'timekeeper': "Gabrielle Peach",
}

new_session = create_session(
    competition_id=new_competition['reference_id'],
    session_datetime=session_data['session_datetime'],
    referee_first=session_data['referee_first'],
    referee_second=session_data['referee_second'],
    referee_third=session_data['referee_third'],
    technical_controller=session_data['technical_controller'],
    jury=session_data['jury'],
    marshall=session_data['marshall'],
    announcer=session_data['announcer'],
    timekeeper=session_data['timekeeper'],
)

session_id = new_session["reference_id"]

Session at 2018-09-21T15:00:00+12:00 created.


In [28]:
weight_category = "W63"
start = 4
end = 17
# session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[start:end]:
    print(lift[1])
if input("Test? [Y]") == "N":
    for lift in lifts[start:end]:
        create_athlete_lift(
            competition_id=competition_id,
            session_id=session_id,
            lift=lift,
            weight_category=weight_category,
            )

Paule Poulin
Sienna Fesolai
Emma McIntyre
Isobel Ferguson
Elizabeth Bateman
Samara Wright
Amber Bailey
Ella Reshef
Rebecca Hall
Tessa Ewens
Amanda Gould
Jodie Mason
Ruby Straugheir
athlete created Paule Poulin
{'athlete_name': 'Paule Poulin', 'snatch_first': 'LIFT', 'snatch_first_weight': 55, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 57, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 58, 'cnj_first': 'NOLIFT', 'cnj_first_weight': 75, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 75, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 75, 'bodyweight': 61.2, 'weight_category': 'W63', 'team': 'GUEST', 'lottery_number': 1}
8rol7AX
Lift created successfully
athlete created Sienna Fesolai
{'athlete_name': 'Sienna Fesolai', 'snatch_first': 'LIFT', 'snatch_first_weight': 60, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 63, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 65, 'cnj_first': 'LIFT', 'cnj_first_weight': 76, 'cnj_second': 'LIFT', 'cnj_second_weight': 79, 'cnj_third': 'LIFT',

In [31]:
df = sessions["Session 4"]
df

,2018 National Championships - Session 3 to 10,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Held:,2018-09-22 00:00:00,NaN,Session #,4,NaN,NaN,at:,"Kolmar, Papatoetoe, Auckland",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Start,NaN,NaN,NaN,Body,SNATCH,NaN,NaN,CLEAN & JERK,NaN,NaN,Best,NaN,NaN,Sinclair,NaN
2,#,Name,Born,Club,weight,1,2,3,1,2,3,Snatch,C & J,TOTAL,Score,Place
3,1,James Kerse,1999,PLUS64,75.6,92,98,-103,117,121,-127,98,121,219,276.088249,7
4,2,Donovan Rea,2000,NSOW,76.2,81,84,88,105,109,-118,88,109,197,247.280777,9
5,3,Ethan Hawken,1996,SOW,73.5,104,108,-112,126,-133,133,108,133,241,308.647953,2
6,4,Anthony Taylor,1996,SHQ,76.2,105,110,-113,132,136,140,110,140,250,313.808093,1
7,5,Roy Johansson,2001,FSO,77,90,94,-98,101,-107,-109,94,101,195,243.389914,10
8,6,Aloysius Lee,1997,SHQ,76.3,91,94,97,118,124,128,97,128,225,282.225677,5
9,7,Oliver Clinton,2001,BOP,74.1,85,93,98,110,120,-128,98,120,218,277.910516,8


In [33]:
session_data = {
    'session_datetime': "2018-09-22T08:00:00+12:00",
    'referee_first': "Tony Ebert",
    'referee_second': "Garry Marshall",
    'referee_third': "Barbara Grieve",
    'technical_controller': "Johan Handwerk",
    'jury': "None",
    'marshall': "Matt Dyson",
    'announcer': "Joe Pomelile",
    'timekeeper': "Joe Pomelile",
}

new_session = create_session(
    competition_id=new_competition['reference_id'],
    session_datetime=session_data['session_datetime'],
    referee_first=session_data['referee_first'],
    referee_second=session_data['referee_second'],
    referee_third=session_data['referee_third'],
    technical_controller=session_data['technical_controller'],
    jury=session_data['jury'],
    marshall=session_data['marshall'],
    announcer=session_data['announcer'],
    timekeeper=session_data['timekeeper'],
)

session_id = new_session["reference_id"]

Session at 2018-09-22T08:00:00+12:00 created.


In [36]:
weight_category = "M77"
start = 3
end = 15
# session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[start:end]:
    print(lift[1])
if input("Test? [Y]") == "N":
    for lift in lifts[start:end]:
        create_athlete_lift(
            competition_id=competition_id,
            session_id=session_id,
            lift=lift,
            weight_category=weight_category,
            )

James Kerse
Donovan Rea
Ethan Hawken
Anthony Taylor
Roy Johansson
Aloysius Lee
Oliver Clinton
Declan Ward
Vester Villalon
David Timajo
Andrew Rodgers
Jack Mallinson
athlete created James Kerse
{'athlete_name': 'James Kerse', 'snatch_first': 'LIFT', 'snatch_first_weight': 92, 'snatch_second': 'LIFT', 'snatch_second_weight': 98, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 103, 'cnj_first': 'LIFT', 'cnj_first_weight': 117, 'cnj_second': 'LIFT', 'cnj_second_weight': 121, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 127, 'bodyweight': 75.6, 'weight_category': 'M77', 'team': 'PLUS64', 'lottery_number': 1}
wNz2WNL
Lift created successfully
athlete created Donovan Rea
{'athlete_name': 'Donovan Rea', 'snatch_first': 'LIFT', 'snatch_first_weight': 81, 'snatch_second': 'LIFT', 'snatch_second_weight': 84, 'snatch_third': 'LIFT', 'snatch_third_weight': 88, 'cnj_first': 'LIFT', 'cnj_first_weight': 105, 'cnj_second': 'LIFT', 'cnj_second_weight': 109, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 118,

In [37]:
df = sessions["Session 5"]
df

,2018 National Championships - Session 3 to 10,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Held:,2018-09-22 00:00:00,NaN,Session #,5,NaN,NaN,at:,"Kolmar, Papatoetoe, Auckland",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Start,NaN,NaN,NaN,Body,SNATCH,NaN,NaN,CLEAN & JERK,NaN,NaN,Best,NaN,NaN,Sinclair,NaN
2,#,Name,Born,Club,weight,1,2,3,1,2,3,Snatch,C & J,TOTAL,Score,Place
3,1,Nikita Parker,1998,OWL,68.5,63,-66,67,-77,78,80,67,80,147,183.554861,10
4,2,Ashleigh Barnes,1991,41DW,68.6,-59,-59,-59,---,---,---,---,---,NaN,NaN,NaN
5,3,Kaeley Che Elkington,1998,WWC,68.9,74,77,80,94,-98,99,80,99,179,222.800607,2
6,4,Cristina Young,1992,INDEP,66.9,60,64,-68,75,80,85,64,85,149,188.521403,9
7,5,Bailey Lovett,1993,EPS,67.4,79,83,-86,98,101,-105,83,101,184,231.830462,1
8,6,Holly Drain,1998,FSO,68.8,71,-75,-77,87,-91,-92,71,87,158,196.818101,7
9,7,Piper Gwyn,2000,UHWC,69,67,71,75,95,98,-105,75,98,173,215.162189,4


In [38]:
session_data = {
    'session_datetime': "2018-09-22T15:00:00+12:00",
    'referee_first': "Odette Smith",
    'referee_second': "Paule Poulin",
    'referee_third': "Jenni Ebert",
    'technical_controller': "Florence Alesana",
    'jury': "None",
    'marshall': "Keisha-Dean Soffee",
    'announcer': "Andy Rodgers",
    'timekeeper': "Alex Stewart"
}

new_session = create_session(
    competition_id=new_competition['reference_id'],
    session_datetime=session_data['session_datetime'],
    referee_first=session_data['referee_first'],
    referee_second=session_data['referee_second'],
    referee_third=session_data['referee_third'],
    technical_controller=session_data['technical_controller'],
    jury=session_data['jury'],
    marshall=session_data['marshall'],
    announcer=session_data['announcer'],
    timekeeper=session_data['timekeeper'],
)

session_id = new_session["reference_id"]

Session at 2018-09-22T15:00:00+12:00 created.


In [42]:
weight_category = "W69"
start = 3
end = 17
# session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[start:end]:
    print(lift[1])
if input("Test? [Y]") == "N":
    for lift in lifts[start:end]:
        create_athlete_lift(
            competition_id=competition_id,
            session_id=session_id,
            lift=lift,
            weight_category=weight_category,
            )

Nikita Parker
Ashleigh Barnes
Kaeley Che Elkington
Cristina Young
Bailey Lovett
Holly Drain
Piper Gwyn
Kelsie Chapman
Samantha Hansen
Nicole Stowers
Alyssa Roy
Sophie Mitchell-Findlater
Jordan Jones-Williams
Megan Signal
athlete created Nikita Parker
{'athlete_name': 'Nikita Parker', 'snatch_first': 'LIFT', 'snatch_first_weight': 63, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 66, 'snatch_third': 'LIFT', 'snatch_third_weight': 67, 'cnj_first': 'NOLIFT', 'cnj_first_weight': 77, 'cnj_second': 'LIFT', 'cnj_second_weight': 78, 'cnj_third': 'LIFT', 'cnj_third_weight': 80, 'bodyweight': 68.5, 'weight_category': 'W69', 'team': 'OWL', 'lottery_number': 1}
0AbyBrx
Lift created successfully
athlete created Ashleigh Barnes
{'athlete_name': 'Ashleigh Barnes', 'snatch_first': 'NOLIFT', 'snatch_first_weight': 59, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 59, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 59, 'cnj_first': 'DNA', 'cnj_first_weight': 0, 'cnj_second': 'DNA', 'cnj_secon

In [43]:
df = sessions["Session 6"]
df

,2018 National Championships - Session 3 to 10,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Held:,2018-09-22 00:00:00,NaN,Session #,6,NaN,NaN,at:,"Kolmar, Papatoetoe, Auckland",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Start,NaN,NaN,NaN,Body,SNATCH,NaN,NaN,CLEAN & JERK,NaN,NaN,Best,NaN,NaN,Sinclair,NaN
2,#,Name,Born,Club,weight,1,2,3,1,2.0,3.0,Snatch,C & J,TOTAL,Score,Place
3,1,Tavite Tonga,2000,POWC,83.6,80,84,88,105,-110.0,110.0,88,110,198,236.9593,12
4,2,Robert Meleisea,1996,POWC,82.5,95,100,105,125,130.0,135.0,105,135,240,289.088659,9
5,3,Cameron McTaggart,1997,FSO,83.4,-123,123,-126,145,-150.0,-150.0,123,145,268,321.105725,4
6,4,Bayley Garnham,2000,NSOW,83.5,83,-87,87,110,-115.0,-115.0,87,110,197,235.899371,13
7,5,Hayden Pritchard,1988,UCOL,84,105,-110,-112,137,-142.0,-146.0,105,137,242,288.950215,8
8,6,Tyler Creighton,2001,BOP,80.6,85,90,95,105,110.0,115.0,95,115,210,255.920529,11
9,7,Albert Delorino,1991,SHQ,81.9,98,103,-107,127,133.0,-138.0,103,133,236,285.300368,10


In [44]:
session_data = {
    'session_datetime': "2018-09-22T16:00:00+12:00",
    'referee_first': "Grant Cavit",
    'referee_second': "Jason Fanning",
    'referee_third': "Andy Jameson",
    'technical_controller': "Barbara Grieve",
    'jury': "None",
    'marshall': "Jodie Mason",
    'announcer': "Declan Ward",
    'timekeeper': "Gabrielle Peach"
}

new_session = create_session(
    competition_id=new_competition['reference_id'],
    session_datetime=session_data['session_datetime'],
    referee_first=session_data['referee_first'],
    referee_second=session_data['referee_second'],
    referee_third=session_data['referee_third'],
    technical_controller=session_data['technical_controller'],
    jury=session_data['jury'],
    marshall=session_data['marshall'],
    announcer=session_data['announcer'],
    timekeeper=session_data['timekeeper'],
)

session_id = new_session["reference_id"]

Session at 2018-09-22T16:00:00+12:00 created.


In [47]:
weight_category = "M85"
start = 3
end = 17
# session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[start:end]:
    print(lift[1])
if input("Test? [Y]") == "N":
    for lift in lifts[start:end]:
        create_athlete_lift(
            competition_id=competition_id,
            session_id=session_id,
            lift=lift,
            weight_category=weight_category,
            )

Tavite Tonga
Robert Meleisea
Cameron McTaggart
Bayley Garnham
Hayden Pritchard
Tyler Creighton
Albert Delorino
Colin Klenner
Uaealesi Funaki
Andrew Pineda
Isaac Lawgun
Timo Hinz
Aiden Te Wake Hope
Cameron Smith
athlete created Tavite Tonga
{'athlete_name': 'Tavite Tonga', 'snatch_first': 'LIFT', 'snatch_first_weight': 80, 'snatch_second': 'LIFT', 'snatch_second_weight': 84, 'snatch_third': 'LIFT', 'snatch_third_weight': 88, 'cnj_first': 'LIFT', 'cnj_first_weight': 105, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 110, 'cnj_third': 'LIFT', 'cnj_third_weight': 110, 'bodyweight': 83.6, 'weight_category': 'M85', 'team': 'POWC', 'lottery_number': 1}
0Aby5rx
Lift created successfully
athlete created Robert Meleisea
{'athlete_name': 'Robert Meleisea', 'snatch_first': 'LIFT', 'snatch_first_weight': 95, 'snatch_second': 'LIFT', 'snatch_second_weight': 100, 'snatch_third': 'LIFT', 'snatch_third_weight': 105, 'cnj_first': 'LIFT', 'cnj_first_weight': 125, 'cnj_second': 'LIFT', 'cnj_second_weight':

In [48]:
df = sessions["Session 7"]
df

,2018 National Championships - Session 3 to 10,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Held:,2018-09-22 00:00:00,NaN,Session #,7,NaN,NaN,at:,"Kolmar, Papatoetoe, Auckland",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Start,NaN,NaN,NaN,Body,SNATCH,NaN,NaN,CLEAN & JERK,NaN,NaN,Best,NaN,NaN,Sinclair,NaN
2,#,Name,Born,Club,weight,1,2,3,1,2.0,3.0,Snatch,C & J,TOTAL,Score,Place
3,1,Jessica Asi,1996,POWC,72.2,-65,65,68,80,-85.0,87.0,68,87,155,188.203794,2
4,2,Eliza Lloyd-Jones,1991,UHWC,71.9,64,-67,69,80,-85.0,-87.0,69,80,149,181.30716,3
5,3,Shanaya Allan,1990,OWL,74,58,62,66,70,75.0,-80.0,66,75,141,169.086342,4
6,4,Matanoko McDonald,2000,KIWI,75,55,61,-68,65,71.0,75.0,61,75,136,162.015409,5
7,5,Bailey Rogers,1991,POWC,74.6,-90,90,-94,-118,-118.0,-118.0,90,---,NaN,NaN,NaN
8,6,Barbra Auva'a,1993,INDEP,73.3,71,-75,-75,-90,-90.0,90.0,71,90,161,193.990778,1
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:

session_data = {
    'session_datetime': "2018-09-22T17:00:00+12:00",
    'referee_first': "Barbara Grieve",
    'referee_second': "Jenni Ebert",
    'referee_third': "Garry Marshall",
    'technical_controller': "Paule Poulin",
    'jury': "None",
    'marshall': "Keisha-Dean Soffee",
    'announcer': "Aisake Tuitupou",
    'timekeeper': "Alex Stewart"
}

new_session = create_session(
    competition_id=new_competition['reference_id'],
    session_datetime=session_data['session_datetime'],
    referee_first=session_data['referee_first'],
    referee_second=session_data['referee_second'],
    referee_third=session_data['referee_third'],
    technical_controller=session_data['technical_controller'],
    jury=session_data['jury'],
    marshall=session_data['marshall'],
    announcer=session_data['announcer'],
    timekeeper=session_data['timekeeper'],
)

session_id = new_session["reference_id"]

Session at 2018-09-22T17:00:00+12:00 created.


In [51]:

weight_category = "W75"
start = 3
end = 9
# session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[start:end]:
    print(lift[1])
if input("Test? [Y]") == "N":
    for lift in lifts[start:end]:
        create_athlete_lift(
            competition_id=competition_id,
            session_id=session_id,
            lift=lift,
            weight_category=weight_category,
            )

Jessica Asi
Eliza Lloyd-Jones
Shanaya Allan
Matanoko McDonald 
Bailey Rogers
Barbra Auva'a
athlete created Jessica Asi
{'athlete_name': 'Jessica Asi', 'snatch_first': 'NOLIFT', 'snatch_first_weight': 65, 'snatch_second': 'LIFT', 'snatch_second_weight': 65, 'snatch_third': 'LIFT', 'snatch_third_weight': 68, 'cnj_first': 'LIFT', 'cnj_first_weight': 80, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 85, 'cnj_third': 'LIFT', 'cnj_third_weight': 87, 'bodyweight': 72.2, 'weight_category': 'W75', 'team': 'POWC', 'lottery_number': 1}
mN4okAx
Lift created successfully
athlete created Eliza Lloyd-Jones
{'athlete_name': 'Eliza Lloyd-Jones', 'snatch_first': 'LIFT', 'snatch_first_weight': 64, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 67, 'snatch_third': 'LIFT', 'snatch_third_weight': 69, 'cnj_first': 'LIFT', 'cnj_first_weight': 80, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 85, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 87, 'bodyweight': 71.9, 'weight_category': 'W75', 'team': 'UHWC', 'lot

In [52]:
# Session 8

df = sessions["Session 8"]
df

,2018 National Championships - Session 3 to 10,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Held:,2018-09-23 00:00:00,NaN,Session #,8,NaN,NaN,at:,"Kolmar, Papatoetoe, Auckland",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Start,NaN,NaN,NaN,Body,SNATCH,NaN,NaN,CLEAN & JERK,NaN,NaN,Best,NaN,NaN,Sinclair,NaN
2,#,Name,Born,Club,weight,1,2,3,1,2.0,3.0,Snatch,C & J,TOTAL,Score,Place
3,1,Joshua Homersham,1998,OWL,86,105,-110,110,135,140.0,-149.0,110,140,250,295.194322,9
4,2,Jaydon Buckley,1995,BOP,93.8,120,-125,-126,143,-151.0,-154.0,120,143,263,298.968987,7
5,3,Robert Jordan,1986,OWL,93.9,-125,125,130,157,-166.0,-166.0,130,157,287,326.10923,2
6,4,James Drayton,1991,INDEP,92.7,112,-116,-117,145,150.0,-156.0,112,150,262,299.286957,8
7,5,Jerome Haretuku,1987,POWC,86.8,100,105,110,125,135.0,-140.0,110,135,245,288.053205,10
8,6,Christian Jessop,1994,INDEP,92.4,123,127,-131,145,-152.0,155.0,127,155,282,322.570019,3
9,7,Lee Attrill,1968,CSI,85.5,97,101,-103,120,-125.0,125.0,101,125,226,267.584575,12


In [53]:
session_data = {
    'session_datetime': "2018-09-23T08:00:00+12:00",
    'referee_first': "Matt Dyson",
    'referee_second': "Jason Fanning",
    'referee_third': "Johan Handwerk",
    'technical_controller': "Johan Handwerk",
    'jury': "None",
    'marshall': "Richie Wong",
    'announcer': "Keisha-Dean Soffee",
    'timekeeper': "Nikita Parker"
}

new_session = create_session(
    competition_id=new_competition['reference_id'],
    session_datetime=session_data['session_datetime'],
    referee_first=session_data['referee_first'],
    referee_second=session_data['referee_second'],
    referee_third=session_data['referee_third'],
    technical_controller=session_data['technical_controller'],
    jury=session_data['jury'],
    marshall=session_data['marshall'],
    announcer=session_data['announcer'],
    timekeeper=session_data['timekeeper'],
)

session_id = new_session["reference_id"]

Session at 2018-09-23T08:00:00+12:00 created.


In [57]:

weight_category = "M94"
start = 3
end = 15
# session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[start:end]:
    print(lift[1])
if input("Test? [Y]") == "N":
    for lift in lifts[start:end]:
        create_athlete_lift(
            competition_id=competition_id,
            session_id=session_id,
            lift=lift,
            weight_category=weight_category,
            )

Joshua Homersham
Jaydon Buckley
Robert Jordan
James Drayton
Jerome Haretuku
Christian Jessop
Lee Attrill
Douglas Sekone-Fraser
Jake Bassett
Stefan Hefer
Patrick Pasia
Montel Tivoli
athlete created Joshua Homersham
{'athlete_name': 'Joshua Homersham', 'snatch_first': 'LIFT', 'snatch_first_weight': 105, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 110, 'snatch_third': 'LIFT', 'snatch_third_weight': 110, 'cnj_first': 'LIFT', 'cnj_first_weight': 135, 'cnj_second': 'LIFT', 'cnj_second_weight': 140, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 149, 'bodyweight': 86.0, 'weight_category': 'M94', 'team': 'OWL', 'lottery_number': 1}
2rdlOrw
Lift created successfully
athlete created Jaydon Buckley
{'athlete_name': 'Jaydon Buckley', 'snatch_first': 'LIFT', 'snatch_first_weight': 120, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 125, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 126, 'cnj_first': 'LIFT', 'cnj_first_weight': 143, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 151, 'cnj_th

In [58]:
df = sessions["Session 9"]
df

,2018 National Championships - Session 3 to 10,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Held:,2018-09-23 00:00:00,NaN,Session #,9,NaN,NaN,at:,"Kolmar, Papatoetoe, Auckland",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Start,NaN,NaN,NaN,Body,SNATCH,NaN,NaN,CLEAN & JERK,NaN,NaN,Best,NaN,NaN,Sinclair,NaN
2,#,Name,Born,Club,weight,1,2,3,1,2.0,3.0,Snatch,C & J,TOTAL,Score,Place
3,NaN,Women 90kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Aotea Frandi,1996,OBC,81.7,66,-70,70,85,90.0,-95.0,70,90,160,183.26575,5
5,2,Kanah Andrews-Nahu,2001,FSO,81.2,88,-93,96,104,-107.0,111.0,96,111,207,237.729607,1
6,3,Mabel Davies,2003,UHWC,87.5,53,56,60,75,-81.0,81.0,60,81,141,157.06217,8
7,4,Zoe Glasson,1991,FSO,77.1,-78,78,-82,-100,-100.0,-100.0,78,---,NaN,NaN,NaN
8,5,Hayley Whiting,1993,CCW,80.4,84,88,92,103,-109.0,109.0,92,109,201,231.84079,2
9,6,Gabrielle Peach,1987,FSO,86.6,69,72,75,86,-89.0,-90.0,75,86,161,180.058962,4


In [59]:
session_data = {
    'session_datetime': "2018-09-23T11:00:00+12:00",
    'referee_first': "Babara Grieve",
    'referee_second': "Jenni Ebert",
    'referee_third': "Keisha Dean-Soffee",
    'technical_controller': "Amanda Gould",
    'jury': "None",
    'marshall': "Jodie Mason",
    'announcer': "Aisake Tuitupou",
    'timekeeper': "Ruby Garbutt"
}

new_session = create_session(
    competition_id=new_competition['reference_id'],
    session_datetime=session_data['session_datetime'],
    referee_first=session_data['referee_first'],
    referee_second=session_data['referee_second'],
    referee_third=session_data['referee_third'],
    technical_controller=session_data['technical_controller'],
    jury=session_data['jury'],
    marshall=session_data['marshall'],
    announcer=session_data['announcer'],
    timekeeper=session_data['timekeeper'],
)

session_id = new_session["reference_id"]

Session at 2018-09-23T11:00:00+12:00 created.


In [65]:
weight_category = "W90"
start = 4
end = 13
# session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[start:end]:
    print(lift[1])
if input("Test? [Y]") == "N":
    for lift in lifts[start:end]:
        create_athlete_lift(
            competition_id=competition_id,
            session_id=session_id,
            lift=lift,
            weight_category=weight_category,
            )

Aotea Frandi
Kanah Andrews-Nahu
Mabel Davies
Zoe Glasson
Hayley Whiting
Gabrielle Peach
Makare Tavanavanua
Sophie Yandall
Elaina Moulds
athlete already exists
{'athlete_name': 'Aotea Frandi', 'snatch_first': 'LIFT', 'snatch_first_weight': 66, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 70, 'snatch_third': 'LIFT', 'snatch_third_weight': 70, 'cnj_first': 'LIFT', 'cnj_first_weight': 85, 'cnj_second': 'LIFT', 'cnj_second_weight': 90, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 95, 'bodyweight': 81.7, 'weight_category': 'W90', 'team': 'OBC', 'lottery_number': 1}
0r666rE
Lift created successfully
athlete already exists
{'athlete_name': 'Kanah Andrews-Nahu', 'snatch_first': 'LIFT', 'snatch_first_weight': 88, 'snatch_second': 'NOLIFT', 'snatch_second_weight': 93, 'snatch_third': 'LIFT', 'snatch_third_weight': 96, 'cnj_first': 'LIFT', 'cnj_first_weight': 104, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 107, 'cnj_third': 'LIFT', 'cnj_third_weight': 111, 'bodyweight': 81.2, 'weight_categor

In [66]:
weight_category = "W90+"
start = 14
end = 19
# session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[start:end]:
    print(lift[1])
if input("Test? [Y]") == "N":
    for lift in lifts[start:end]:
        create_athlete_lift(
            competition_id=competition_id,
            session_id=session_id,
            lift=lift,
            weight_category=weight_category,
            )

Laurel Hubbard
Miniah Summerell
Angel-Leigh Durham
Jessica Collins
Jessica Moran
athlete created Laurel Hubbard
{'athlete_name': 'Laurel Hubbard', 'snatch_first': 'LIFT', 'snatch_first_weight': 100, 'snatch_second': 'LIFT', 'snatch_second_weight': 105, 'snatch_third': 'LIFT', 'snatch_third_weight': 110, 'cnj_first': 'NOLIFT', 'cnj_first_weight': 122, 'cnj_second': 'LIFT', 'cnj_second_weight': 122, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 130, 'bodyweight': 141.2, 'weight_category': 'W90+', 'team': 'OWL', 'lottery_number': 10}
gNmzZYa
Lift created successfully
athlete already exists
{'athlete_name': 'Miniah Summerell', 'snatch_first': 'LIFT', 'snatch_first_weight': 60, 'snatch_second': 'LIFT', 'snatch_second_weight': 64, 'snatch_third': 'LIFT', 'snatch_third_weight': 67, 'cnj_first': 'LIFT', 'cnj_first_weight': 70, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 74, 'cnj_third': 'LIFT', 'cnj_third_weight': 78, 'bodyweight': 101.3, 'weight_category': 'W90+', 'team': 'INDEP', 'lottery_numb

In [67]:
df = sessions["Session 10"]
df

,2018 Junior Senior National Championships,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Held:,2018-09-23 00:00:00,NaN,Session #,10,NaN,NaN,at:,"Kolmar, Papatoetoe, Auckland",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Start,NaN,NaN,NaN,Body,SNATCH,NaN,NaN,CLEAN & JERK,NaN,NaN,Best,NaN,NaN,Sinclair,NaN
2,#,Name,Born,Club,weight,1,2,3,1,2.0,3,Snatch,C & J,TOTAL,Score,Place
3,NaN,Men 105kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Hokianga Morunga,1993,UHWC,99,93,98,-103,-130,130.0,136,98,136,234,260.441755,7
5,2,Sebastian Margaritis,1995,CSI,101.6,102,107,111,125,130.0,-133,111,130,241,265.703257,6
6,3,Andy Barakauskas,1992,CCW,104.2,140,143,-146,173,178.0,-181,143,178,321,350.798228,1
7,4,Aden Zhu,1997,CCW,103.2,-113,113,117,133,138.0,-145,117,138,255,279.596578,5
8,5,Richard Jones,1986,NSOW,104.9,-145,-145,145,175,-180.0,-181,145,175,320,348.914106,2
9,6,Sione Tonga,1992,POWC,94.7,115,120,125,145,151.0,-160,125,151,276,312.532653,3


In [68]:
session_data = {
    'session_datetime': "2018-09-23T14:00:00+12:00",
    'referee_first': "Tony Ebert",
    'referee_second': "Garry Marshall",
    'referee_third': "Barbara Grieve",
    'technical_controller': "None",
    'jury': "None",
    'marshall': "None",
    'announcer': "None",
    'timekeeper': "None"
}

new_session = create_session(
    competition_id=new_competition['reference_id'],
    session_datetime=session_data['session_datetime'],
    referee_first=session_data['referee_first'],
    referee_second=session_data['referee_second'],
    referee_third=session_data['referee_third'],
    technical_controller=session_data['technical_controller'],
    jury=session_data['jury'],
    marshall=session_data['marshall'],
    announcer=session_data['announcer'],
    timekeeper=session_data['timekeeper'],
)

session_id = new_session["reference_id"]

Session at 2018-09-23T14:00:00+12:00 created.


In [72]:
weight_category = "M105"
start = 4
end = 11
# session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[start:end]:
    print(lift[1])
if input("Test? [Y]") == "N":
    for lift in lifts[start:end]:
        create_athlete_lift(
            competition_id=competition_id,
            session_id=session_id,
            lift=lift,
            weight_category=weight_category,
            )

Hokianga Morunga
Sebastian Margaritis
Andy Barakauskas
Aden Zhu
Richard Jones
Sione Tonga
Michael Walker
athlete created Hokianga Morunga
{'athlete_name': 'Hokianga Morunga', 'snatch_first': 'LIFT', 'snatch_first_weight': 93, 'snatch_second': 'LIFT', 'snatch_second_weight': 98, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 103, 'cnj_first': 'NOLIFT', 'cnj_first_weight': 130, 'cnj_second': 'LIFT', 'cnj_second_weight': 130, 'cnj_third': 'LIFT', 'cnj_third_weight': 136, 'bodyweight': 99.0, 'weight_category': 'M105', 'team': 'UHWC', 'lottery_number': 1}
vrWJKYp
Lift created successfully
athlete created Sebastian Margaritis
{'athlete_name': 'Sebastian Margaritis', 'snatch_first': 'LIFT', 'snatch_first_weight': 102, 'snatch_second': 'LIFT', 'snatch_second_weight': 107, 'snatch_third': 'LIFT', 'snatch_third_weight': 111, 'cnj_first': 'LIFT', 'cnj_first_weight': 125, 'cnj_second': 'LIFT', 'cnj_second_weight': 130, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 133, 'bodyweight': 101.6, 'weight_

In [75]:
weight_category = "M105+"
start = 12
end = 19
# session_id = "RWKRpDO" # PLEASE SET THIS

lifts = [_[1] for _ in df.iterrows()]
for lift in lifts[start:end]:
    print(lift[1])
if input("Test? [Y]") == "N":
    for lift in lifts[start:end]:
        create_athlete_lift(
            competition_id=competition_id,
            session_id=session_id,
            lift=lift,
            weight_category=weight_category,
            )

Kesniel Setefano
Koale Tasi
Callan Helms
Joachim Setefano
Sio (Joe) Pomelile
Tevita Ngalu
David Liti
athlete created Kesniel Setefano
{'athlete_name': 'Kesniel Setefano', 'snatch_first': 'LIFT', 'snatch_first_weight': 100, 'snatch_second': 'LIFT', 'snatch_second_weight': 105, 'snatch_third': 'NOLIFT', 'snatch_third_weight': 110, 'cnj_first': 'LIFT', 'cnj_first_weight': 125, 'cnj_second': 'NOLIFT', 'cnj_second_weight': 130, 'cnj_third': 'LIFT', 'cnj_third_weight': 130, 'bodyweight': 122.5, 'weight_category': 'M105+', 'team': 'POWC', 'lottery_number': 8}
mAe4PrK
Lift created successfully
athlete already exists
{'athlete_name': 'Koale Tasi', 'snatch_first': 'LIFT', 'snatch_first_weight': 130, 'snatch_second': 'LIFT', 'snatch_second_weight': 142, 'snatch_third': 'LIFT', 'snatch_third_weight': 150, 'cnj_first': 'LIFT', 'cnj_first_weight': 170, 'cnj_second': 'LIFT', 'cnj_second_weight': 185, 'cnj_third': 'NOLIFT', 'cnj_third_weight': 200, 'bodyweight': 117.8, 'weight_category': 'M105+', 'tea